<a href="https://colab.research.google.com/github/SamuelXJames/RealRes/blob/main/models/ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ESRGAN
### ISR Implementation

This code trains ESRGAN using LR images from realistic kernels

## Connect to Google Cloud Storage (GCS)

The images are stored on a GCS bucket. Alternatively, they can be stored locally or on Google Drive 



In [ ]:
## IGNORE if storing kerenels locally or on Google Drive
#Authenticare access to cloud storage
!gcloud auth login

#Set current GCS project
!gcloud config set project superres000

In [ ]:
## IGNORE if storing kerenels locally or on Google Drive
#Setup google cloud sdk (seems to be necessary but could be redudant)
from google.colab import auth
auth.authenticate_user()

## Download Data from GCS

The data can also be stored on Google Drive, however, downloading large files from Google Drive can take some time.

In [ ]:
!gsutil -m cp -r "gs://super_res_train/DIV2K_train_LR_Real_X4/" "/content/"
!gsutil -m cp -r "gs://super_res_train/idealo/Images/DIV2K_valid_HR/" "/content/"
!gsutil -m cp -r "gs://super_res_train/DIV2K_valid_LR_Real_X4/" "/content/"
!gsutil -m cp -r "gs://super_res_train/idealo/Images/DIV2K_train_HR/" "/content/"

## Install Requirements
Tensorflow-gpu  is installed as otherwise the gpu is not recognized. 

In [ ]:
# Install requirements
!pip install ISR
!pip install wandb
!pip uninstall tensorflow
!pip install tensorflow-gpu

In [ ]:
# This is used to verify the GPU is being utilized
import wandb
wandb.init()

## Create ESRGAN Model

In [ ]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

lr_train_patch_size = 40
layers_to_extract = [5, 9]
scale = 4
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':32, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

# Needs to be updated each time weights are being reloaded
gen_weights = None
discr_weights = None
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
}

log_dirs = {'logs': '/content/drive/MyDrive/idealo/x4/logs', 'weights': '/content/drive/MyDrive/idealo/x4/weights'}

learning_rate = {'initial_value': 0.0008, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir='/content/DIV2K_train_LR_Real_X4',
    hr_train_dir='/content/DIV2K_train_HR',
    lr_valid_dir='/content/DIV2K_valid_LR_Real_X4',
    hr_valid_dir='/content/DIV2K_valid_HR',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='image_dataset',
    log_dirs=log_dirs,
    weights_generator=gen_weights,
    weights_discriminator=discr_weights,
    n_validation=100,
)



In [ ]:
!git clone https://github.com/idealo/image-super-resolution.git

Cloning into 'image-super-resolution'...
remote: Enumerating objects: 1157, done.
remote: Total 1157 (delta 0), reused 0 (delta 0), pack-reused 1157
Receiving objects: 100% (1157/1157), 14.94 MiB | 40.69 MiB/s, done.
Resolving deltas: 100% (579/579), done.


## Verify the GPU is connected

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# Setup tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/idealo/logs

## Train Model

In [ ]:
trainer.train(
    epochs=80,
    steps_per_epoch=500,
    batch_size=16,
    monitored_metrics={'val_loss': 'min'}
)

Could not find previous configuration

Training details:
  training_parameters: 
    metrics: {'generator': <function PSNR_Y at 0x7fb871be10d0>}
    losses: {'generator': 'mae', 'discriminator': 'binary_crossentropy', 'feature_extractor': 'mse'}
    adam_optimizer: {'beta1': 0.9, 'beta2': 0.999, 'epsilon': None}
    learning_rate: {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    n_validation: 100
    dataname: image_dataset
    fallback_save_every_n_epochs: 2
    log_dirs: {'logs': '/content/drive/MyDrive/idealo/logs', 'weights': '/content/drive/MyDrive/idealo/weights'}
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    hr_valid_dir: /content/DIV2K_valid_HR
    lr_valid_dir: /content/DIV2K_valid_LR_Real_X2
    hr_train_dir: /content/DIV2K_train_HR
    lr_train_dir: /content/DIV2K_train_LR_Real_X2
    lr_patch_size: 40
    steps_per_epoch:

Epoch 61/80
Current learning rate: 9.999999747378752e-05
Current flatness treshold: 0.12
100%|██████████| 500/500 [09:24<00:00,  1.13s/it]
Epoch 61 took      564.7s


100/100 [==============================] - 19s 191ms/step - loss: 0.3305 - generator_loss: 0.7355 - discriminator_loss: 0.1671 - feature_extractor_loss: 2.0930 - feature_extractor_1_loss: 5.8031 - generator_PSNR_Y: 10.7570


{'val_loss': 0.3305455446243286, 'val_generator_loss': 0.735495924949646, 'val_discriminator_loss': 0.1670534908771515, 'val_feature_extractor_loss': 2.093021869659424, 'val_feature_extractor_1_loss': 5.803133964538574, 'val_generator_PSNR_Y': 10.757011413574219, 'train_d_real_loss': 7.303685656268044e-09, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 6.7876921860943185e-09, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.9489012360572815, 'train_generator_loss': 0.727986752986908, 'train_discriminator_loss': 0.1731143593788147, 'train_feature_extractor_loss': 6.544913291931152, 'train_feature_extractor_1_loss': 16.196264266967773, 'train_generator_PSNR_Y': 10.175929069519043}
val_loss improved from        inf to    0.33055
Saving weights
Epoch 62/80
Current learning rate: 9.999999747378752e-05
Current flatness treshold: 0.12
100%|██████████| 500/500 [09:10<00:00,  1.10s/it]
Epoch 62 took      550.0s


100/100 [==============================] - 19s 191ms/step - loss: 0.3308 - generator_loss: 0.7448 - discriminator_loss: 0.2181 - feature_extractor_loss: 2.0769 - feature_extractor_1_loss: 5.8131 - generator_PSNR_Y: 10.8457


...

{'val_loss': 0.3220360279083252, 'val_generator_loss': 1.3363401889801025, 'val_discriminator_loss': 0.07290718704462051, 'val_feature_extractor_loss': 2.0297482013702393, 'val_feature_extractor_1_loss': 5.684703826904297, 'val_generator_PSNR_Y': 9.498943328857422, 'train_d_real_loss': 3.16729597216181e-06, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 3.833389655483188e-06, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.29575103521347046, 'train_generator_loss': 1.389113187789917, 'train_discriminator_loss': 0.0703640729188919, 'train_feature_extractor_loss': 2.3200416564941406, 'train_feature_extractor_1_loss': 4.76392936706543, 'train_generator_PSNR_Y': 9.0690279006958}
val_loss did not improve.
Epoch 78/80
Current learning rate: 9.999999747378752e-05
Current flatness treshold: 0.15
100%|██████████| 500/500 [09:26<00:00,  1.13s/it]
Epoch 78 took      566.8s


100/100 [==============================] - 19s 195ms/step - loss: 0.3200 - generator_loss: 1.6869 - discriminator_loss: 0.0967 - feature_extractor_loss: 2.0541 - feature_extractor_1_loss: 5.6064 - generator_PSNR_Y: 8.3097


{'val_loss': 0.32002463936805725, 'val_generator_loss': 1.6869137287139893, 'val_discriminator_loss': 0.09667947143316269, 'val_feature_extractor_loss': 2.054095983505249, 'val_feature_extractor_1_loss': 5.606355667114258, 'val_generator_PSNR_Y': 8.309707641601562, 'train_d_real_loss': 2.0764159671671223e-06, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 3.7086053907842143e-06, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.24530011415481567, 'train_generator_loss': 1.6836037635803223, 'train_discriminator_loss': 0.08993662893772125, 'train_feature_extractor_loss': 1.56085205078125, 'train_feature_extractor_1_loss': 4.307113170623779, 'train_generator_PSNR_Y': 8.297277450561523}
val_loss did not improve.
Epoch 79/80
Current learning rate: 9.999999747378752e-05
Current flatness treshold: 0.15
100%|██████████| 500/500 [09:22<00:00,  1.13s/it]
Epoch 79 took      562.8s


100/100 [==============================] - 19s 194ms/step - loss: 0.3245 - generator_loss: 1.8084 - discriminator_loss: 0.0912 - feature_extractor_loss: 2.1279 - feature_extractor_1_loss: 5.6414 - generator_PSNR_Y: 7.8086


{'val_loss': 0.3245009183883667, 'val_generator_loss': 1.808439016342163, 'val_discriminator_loss': 0.09120664745569229, 'val_feature_extractor_loss': 2.1278812885284424, 'val_feature_extractor_1_loss': 5.641359806060791, 'val_generator_PSNR_Y': 7.808628082275391, 'train_d_real_loss': 9.498170925326122e-07, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 3.8043208405724727e-06, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.24681994318962097, 'train_generator_loss': 1.8382354974746704, 'train_discriminator_loss': 0.09516040980815887, 'train_feature_extractor_loss': 1.7055619955062866, 'train_feature_extractor_1_loss': 4.197638988494873, 'train_generator_PSNR_Y': 7.645621299743652}
val_loss did not improve.
Saving weights anyways.
